In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import os
import random

from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import KFold, GroupKFold
from sklearn.utils import shuffle

import cv2
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

import torch
from torch import nn, optim
import pl_bolts as pb
from torch.utils.data import Dataset, DataLoader

import pretrainedmodels as ptm
import geffnet as gf
import timm
import segmentation_models_pytorch as sm

fp16 = True
scaler = torch.cuda.amp.GradScaler(enabled=fp16)

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

n = 0
torch.cuda.set_device(n)

In [ ]:
train_image = pd.read_csv("/mnt/md0/siim_covid/train_image_level.csv")
train_image['id'] = train_image['id'].str.replace("_image", "")
train_image["sid"] = train_image.StudyInstanceUID

meta = pd.read_csv("/mnt/md0/siim_covid/meta.csv").iloc[:6334]
meta["id"] = meta['image_id']
meta = meta.drop(['image_id'], axis=1)

train_image = pd.merge(train_image, meta, on="id")

train_study = pd.read_csv("/mnt/md0/siim_covid/train_study_level.csv")
train_study['id'] = train_study['id'].str.replace("_study", "")
train_study['sid'] = train_study['id']
train_study = train_study.drop(['id'], axis=1)

data = pd.merge(train_image, train_study, on="sid")
data.head()

In [ ]:
class SIIMDataset(Dataset):
    def __init__(self, data, transforms, path):
        self.data = data
        self.transforms = transforms
        self.path = path
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        row = self.data.iloc[i]
        image = cv2.imread(self.path + row['id'] + ".jpg")
        image = (image/255).astype('float32')
        label = row[8:].values.astype('float32')
        bboxes = [row['label'].split(" ")[i:i+6] for i in range(0, 
                    len(self.data.iloc[0]['label'].split(" ")), 6)]
        
        sz = 12
        h_ratio = sz/row['dim0']
        w_ratio = sz/row['dim1']
        st = row['label'].split(" ")
        mask = np.zeros((sz, sz, 1)).astype('float32')
        for s in range(len(st)//6):
            dims = st[s*6:(s+1)*6][2:]
            mask[int(float(dims[1])*h_ratio):int(float(dims[3])*h_ratio),
                 int(float(dims[0])*w_ratio):int(float(dims[2])*w_ratio), :] = 1.
        
        if self.transforms: image = self.transforms(image=image)['image']
        
        label = np.array([row[8]]).astype(np.float32)
        
        return image, label, mask.transpose(2, 0, 1)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    if keep_ratio: return im.thumbnail((size, size), resample)
    else: return im.resize((size, size), resample)
    
class TestDataset(Dataset):
    def __init__(self, data, transforms=None):
        self.data = data
        self.transforms = A.Compose([
            #A.Resize(384, 384),
            ToTensorV2(),
            ])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        image = cv2.imread(f'/mnt/md0/siim_covid/test_png/' + self.data[i])
        image = np.array(resize(image, 384))
        image = (image/255).astype('float32')
        
        if self.transforms: image = self.transforms(image=image)['image']
        
        return image

def create_dataloders(train, test, TRN_BS=32, TST_BS=4, train_augs=None, test_augs=None):
    
    if not train_augs:
        SZ = 384
        train_augs = A.Compose([
            A.Resize(SZ, SZ),
            A.HorizontalFlip(p=0.5),
            ToTensorV2()
        ])
    if not test_augs:
        test_augs = A.Compose([
            A.Resize(SZ, SZ),
            ToTensorV2()
        ])

    train_dataset = SIIMDataset(train, train_augs, "/mnt/md0/siim_covid/train/")
    test_dataset = SIIMDataset(test, test_augs, "/mnt/md0/siim_covid/train/")

    train_loader = DataLoader(train_dataset, batch_size=TRN_BS, shuffle=True, 
                              num_workers=2, worker_init_fn=seed_worker)
    test_loader = DataLoader(test_dataset, batch_size=TST_BS, shuffle=False, 
                             num_workers=2, worker_init_fn=seed_worker)

    return train_loader, test_loader

In [ ]:
train, test = data[:int(len(data)*0.8)], data[int(len(data)*0.8):]

train_loader, test_loader = create_dataloders(train, test)

for d in test_loader: break
plt.imshow(d[0][0].numpy().transpose(1, 2, 0))

In [ ]:
class PcamPool(nn.Module):
    def __init__(self):
        super(PcamPool, self).__init__()

    def forward(self, feat_map, prob_map):
        weight_map = prob_map / prob_map.sum(dim=2, keepdim=True).sum(dim=3, keepdim=True)
        feat = (feat_map * weight_map).sum(dim=2, keepdim=True).sum(dim=3, keepdim=True)
        return feat
    
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        #self.backbone = nn.Sequential(*[c for c in timm.models.tf_efficientnetv2_m_in21ft1k(pretrained=True).children()][:-2])
        self.backbone = timm.models.tf_efficientnetv2_l_in21ft1k(pretrained=True)
        
        self.b0 = nn.Sequential(
            self.backbone.conv_stem,
            self.backbone.bn1,
            self.backbone.act1,
        )
        
        self.b1 = self.backbone.blocks[0]
        self.b2 = self.backbone.blocks[1]
        self.b3 = self.backbone.blocks[2]
        self.b4 = self.backbone.blocks[3]
        
        self.b5 = self.backbone.blocks[4]
        self.b6 = self.backbone.blocks[5]
        self.b7 = self.backbone.blocks[6]
        self.b8 = nn.Sequential(
            self.backbone.conv_head,
            self.backbone.bn2,
            self.backbone.act2,
        )
        
        self.cam_ = nn.Conv1d(1280, 1, (1, 1))
        
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.maxpool = nn.AdaptiveMaxPool2d(1)
        self.pcampool = PcamPool()
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax(-1)
        self.sigmoid = nn.Sigmoid()
        
        f = 1280
        
        self.classifier = nn.Linear(f*2, 1)
    
    def forward(self, inp):
        #feats = self.backbone(inp)
        
        x = self.b0(inp) #torch.Size([4, 24, 192, 192])
        x = self.b1(x) #torch.Size([4, 24, 192, 192])
        x = self.b2(x) #torch.Size([4, 48, 96, 96])
        x = self.b3(x) #torch.Size([4, 80, 48, 48])
        
        #pre_meta = self.flatten(torch.mean(x, axis=1))
        #meta = self.meta(pre_meta)
        
        x = self.b4(x) #torch.Size([4, 160, 24, 24])
        
        x = self.b5(x) #torch.Size([4, 176, 24, 24])
        
        x = self.b6(x) #torch.Size([4, 304, 12, 12])
        x = self.b7(x) #torch.Size([4, 512, 12, 12])
        feats = self.b8(x) # torch.Size([4, 1280, 12, 12])
        
        #sh = feats.shape[2:]
        #cam_ = self.sigmoid(self.seg_backbone(inp))
        #cam = nn.functional.interpolate(cam_, sh)
        cam_ = self.cam_(feats)
        cam = self.sigmoid(cam_)
        #print(feats.shape, cam.shape)
        pcampool = self.flatten(self.pcampool(feats, cam))
        #print(pcampool.shape)
        #feats = self.drop20(feats)
        avgpool = self.flatten(self.avgpool(feats))
        #maxpool = self.flatten(self.maxpool(feats))
        catpool = torch.cat([avgpool, pcampool], -1)
        logits = self.classifier(catpool)
        
        return logits, cam_#torch.zeros((inp.shape[0],)).cuda().half()#cam_

In [ ]:
model = Model().cuda()
#model = nn.DataParallel(Model(), device_ids=[0, 1, 2, 3]).cuda()
o = model(d[0].cuda()) #Not doing for saving ram
try: print(o)
except: print("Not doing for saving ram")

In [ ]:
def train_fn(model, loader, criterion, seg_criterion, optimizer, acm=1):
    model.train()
    
    tq = tqdm(loader)
    
    running_loss, running_auc = 0., 0.
    
    lbls, outs = [], []
    optimizer.zero_grad()
    for i, data in enumerate(tq):
        
        inputs, labels = data[0], data[1]
        inputs = inputs.cuda()
        labels = labels.cuda()
        if fp16:
            inputs = inputs.half()
            labels = labels.half()
        
        with torch.cuda.amp.autocast(enabled=fp16):
            outputs, cams = model(inputs)
            loss = criterion(outputs, labels) + seg_criterion(cams, data[2].cuda())#criterion(torch.sigmoid(outputs), torch.argmax(labels, 1))
        
        scaler.scale(loss).backward()
        if not i % acm:
            scaler.step(optimizer)
            optimizer.zero_grad()
        scaler.update()
        
        running_loss += loss.item()
        labels, outputs = labels.detach().cpu(), outputs.detach().cpu()
        try: running_auc += roc_auc_score(np.concatenate(labels.numpy(), 0), np.concatenate(outputs.numpy(), 0))
        except: running_auc += .5
        
        tq.set_postfix(loss=running_loss/(i+1), auc=running_auc/(i+1))
        
        lbls.append(labels.cpu().detach().numpy())
        outs.append(outputs.cpu().detach().numpy())
    
    return np.concatenate(lbls), np.concatenate(outs), [running_loss/(i+1), running_auc/(i+1)] 

def test_fn(model, loader, criterion, seg_criterion):
    model.eval()
    
    tq = tqdm(loader)
    
    running_loss, running_auc = 0., 0.
    
    lbls, outs = [], []
    
    for i, data in enumerate(tq):
        
        inputs, labels = data[0], data[1]
        inputs = inputs.cuda()
        labels = labels.cuda()
        if fp16:
            inputs = inputs.half()
            labels = labels.half()
        
        with torch.cuda.amp.autocast(enabled=fp16):
            outputs, cams = model(inputs)
            loss = criterion(outputs, labels) + seg_criterion(cams, data[2].cuda())#criterion(torch.sigmoid(outputs), torch.argmax(labels, 1))
        
        running_loss += loss.item()
        labels, outputs = labels.detach().cpu(), outputs.detach().cpu()
        try: running_auc += roc_auc_score(np.concatenate(labels.numpy(), 0), np.concatenate(outputs.numpy(), 0))
        except: running_auc += .5
        
        tq.set_postfix(loss=running_loss/(i+1), auc=running_auc/(i+1))
        
        lbls.append(labels.cpu().detach().numpy())
        outs.append(outputs.cpu().detach().numpy())
        
    return np.concatenate(lbls), np.concatenate(outs), [running_loss/(i+1), running_auc/(i+1)] 

In [ ]:
SIZE = 384

train_augs = A.Compose([
    A.RandomResizedCrop(SIZE, SIZE, scale=(0.3, 1.2)),
    A.HorizontalFlip(p=0.5),
    A.Rotate(p=0.5, limit=45),
    A.RandomBrightnessContrast(p=.5),
    #A.RandomGamma(p=0.5),
    A.Cutout(p=0.25, num_holes=4, max_h_size=64, max_w_size=64),
    
    ToTensorV2()
])

test_augs = A.Compose([
    A.Resize(SIZE, SIZE),
    ToTensorV2()
])

TRN_BS = 32
acm = 1
TST_BS = 4
EPOCHS = 20

scores = []

DIR = "/mnt/md0/siim_covid/models/"
BACKBONE = "effv2l_2cls"
version = "2"

try: os.mkdir(f"{DIR}/{BACKBONE}_v{version}/")
except: pass

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

for i, (d1, d2) in enumerate(GroupKFold(n_splits=5).split(data, groups=data.id.tolist())):
    print(f"FOLD {i}")
    
    model = nn.DataParallel(Model(), device_ids=[n, n+1]).cuda()
    
    curr_best = -np.inf
    
    train, test = data.iloc[d1], data.iloc[d2]
    train_loader, test_loader = create_dataloders(train, test, TRN_BS, TST_BS, train_augs, test_augs)
    
    criterion = nn.BCEWithLogitsLoss()#nn.CrossEntropyLoss
    seg_criterion = nn.BCEWithLogitsLoss()
    
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = pb.optimizers.LinearWarmupCosineAnnealingLR(optimizer,
                                                            warmup_epochs=3, max_epochs=EPOCHS,
                                                            warmup_start_lr=1e-4, eta_min=1e-6)

    for _ in range(EPOCHS):
        print(f"TRAINING EPOCH {_} LR: {get_lr(optimizer)}")
        trn_lbls, trn_outs, metrics = train_fn(model, train_loader, criterion, seg_criterion, optimizer, acm)
        print(f"TESTING EPOCH {_}")
        tst_lbls, tst_outs, metrics = test_fn(model, test_loader, criterion, seg_criterion)
        score = average_precision_score(tst_lbls[:, 0], tst_outs[:, 0]) * 1/3
        
        if score > curr_best:
            print("NEW BEST: {:.3f}".format(score))
            best_state = model
            curr_best = score
        
        scheduler.step()
    
    scores.append(curr_best)
    
    
    try: torch.jit.save(torch.jit.script(best_state.module), f"{DIR}/{BACKBONE}_v{version}/f{i}.pth")
    except:
        try: torch.save(best_state.module.state_dict(), f"{DIR}/{BACKBONE}_v{version}/f{i}_st.pth")
        except: print("Not Saved")
    open(f"{DIR}/{BACKBONE}_v{version}/f{i}.txt", "w").write(str(curr_best))
    
    sub = pd.read_csv("/mnt/md0/siim_covid/updated_sample_submission.csv")
    sub.dcm_path = sub['dcm_path'].str.replace("/kaggle/input/siim-covid19-detection/test/", "").str.split("/")
    
    images = [p[-1].replace('.dcm', '.png') for p in sub.dcm_path[1214:]]
    test_dataset = TestDataset(images)
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=4)
    preds = []
    for d in tqdm(test_loader):
        outs = []
        with torch.no_grad():
                #o = torch.sigmoid(m(d.cuda())).detach().cpu().numpy()
            o = torch.sigmoid(best_state(d.cuda())[0]).detach().cpu().numpy()
            outs.append(o)
        outs = np.mean(outs, 0)
        preds.append(outs)
    preds = np.concatenate(preds)
    
    sub_df = sub.copy()
    sub = sub[1214:]
    sub = sub.reset_index()
    sub['none'] = 0
    sub[['none']] = preds
    for j in range(1263):
        none = sub.loc[j,'none']
        sub.loc[j, 'PredictionString'] = f'none {none} 0 0 1 1'

    sub_df = sub_df[:1214]
    sub_df = sub_df.append(sub).reset_index()[['id', 'PredictionString']]

    sub_df.to_csv(f"{DIR}/{BACKBONE}_v{version}/f{i}_sub.csv", index=False)

In [ ]:
scores

In [ ]:
DIR = "/mnt/md0/siim_covid/models/"
BACKBONE = "effv2l_2cls"
version = '2'
subs = [pd.read_csv(f"{DIR}/{BACKBONE}_v{version}/{f}") for f in os.listdir(f"{DIR}/{BACKBONE}_v{version}/") if f.endswith("_sub.csv")]
labels = []
for sub in subs:
    cache = []
    for _ in range(len(preds)):
        predstring = sub.loc[1214+_].PredictionString.split(" ")
        cache.append([predstring[1]])
    labels.append(cache)
labels = np.mean(np.array(labels).astype(np.float32), 0)

sub = pd.read_csv("/mnt/md0/siim_covid/updated_sample_submission.csv")

sub_df = sub.copy()
sub = sub[1214:]

for j in tqdm(range(len(preds))):
    none = labels[j][0]
    sub.loc[1214+j, 'PredictionString'] = f'none {none} 0 0 1 1'

sub_df[1214:] = sub
sub_df =  sub_df[['id', 'PredictionString']]
#sub = sub.append(sub_df[1214:])[['id', 'PredictionString']]
    
sub_df.to_csv(f"{DIR}/{BACKBONE}_v{version}/none_sub.csv", index=False)

In [ ]:
sub_df